# TOEFLやTOEICのスコアリングについて考える

TOELEやTOEICは世界的（後者は日本国内で主流）な英語能力の検定試験の一種です。大学入試や留学，就活などでこれら検定試験のスコアが活用されることもあります。

ところで，TOEICって，そのスコアのレンジ（範囲）が10点〜990点になるように **統計的な処理** が施されています。その理由はいくつかありますが，大きなものとしては「複数回受験した場合でも，能力を比較可能な形で表現するため」だったり，「異なる回を受験した，異なる受験者のスコアをなるべく平等に比較するため」だったりします。

この**統計的な処理**については様々な憶測がありますが，よくあるのは「素点をもとに偏差値的な処理をする」というなんとも腑に落ちないものです。公式問題集には素点とスコアのざっくりとした対応表がついており，そこから，「50門正答したら，大体hoge点 ~ fuga点の間」ということがわかるようですが，やはり，そのスコアリングについては触れられていません。

「TOEIC　スコア　計算」などで検索すると，上記のようなことを説明してくれる不親切なまとめサイトがゴロゴロある一方で，みんなが口を揃えて言う**統計的な処理**については殆ど触れられていません。それもそのはずで，TOEICのテストを運用，スコアリングしているETSという非営利団体は，このスコアの計算方法を詳細に公開していないからです（のちに触れますが，これは決して受験者に意地悪しているのではなく，公平なテスト運用に欠かせないのです）。

## TOEICのスコアリングに用いる統計処理について

TOEICのスコアリングの方法は，実際のところETSが公開していないので，わかりません。これにつきます。[Examinee Handbook (直PDF)](https://www.ets.org/s/toeic/pdf/listening-reading-examinee-handbook.pdf)によれば，

- Scores are determined by the number of correct answers, which is converted to a scaled score. 
- Scaled scores are transformed and derived from test takers’ raw scores through a proven statistical procedure called “equating.” 

とだけ記されています。つまり，正答数をなんらかの変換を施したものがスコアであり，そのスコア算出にはequatingという処理が施されている，ということです。

よく「TOEICにはIRT(項目反応理論)が使われている」と言われるのですが，そう**予想**されているだけです（私もそうだと思っていて，まず間違いなくIRTを使っていると思っています。その理由については後述）。

＃＃ TOEFLのスコアリングに用いる統計処理について
TOEFLに関しては，IRTが使われていることが明言されています。たとえばこの[論文](https://onlinelibrary.wiley.com/doi/abs/10.1002/j.2333-8504.1993.tb01570.x)の初っ端に「IRTの3PLMっての使ってるで」って書いてあります。

## 項目反応理論とはなにものなのか

項目反応理論とは，一言でいうと，「個人の能力レベルと項目ごとの難しさなどを考慮して，個人が特定の項目に正答できる確率をモデル化したもの」です。具体的には，

$$
P(\theta) = \gamma + \frac{1-\gamma}{1 + exp(\alpha(\theta - \beta))}
$$

このようなモデルに基づいています。$\alpha, \beta, \gamma$はそれぞれ項目の識別力，困難度，当て推量のパラメタであり，$\theta$が受験者の能力値を表しています。この$\theta$に基づいてTOEFLのスコアは計算されています。IRTではこの反応確率を$\theta$の関数とみなしたものをICC(Item Characteristic Curve)と呼んでいます。

この$\theta$が重要で，理論上は$[-\infty, \infty]$の区間の任意の値を取ります。しかも困ったことに，$\theta$と$\beta$を線形変換すると，パラメタは異なるのに正当確率が全く同じという組み合わせが無限にできてしまいます。これを尺度の不定性などと呼びます。この尺度の不定性を解決するために，IRTでは一般に「$\theta$の平均を0，標準偏差を1」に固定して，すべてのパラメタを推定します。そうすると，受験者集団の$\theta$が標準正規分布に従っていれば，プラスマイナス4標準偏差の範囲に，ほぼ100％の受験者のスコアが入ってくることになります。すべてのパラメタが推定し終わった後に，$\theta$を任意の平均と標準偏差になるように線形変換すると，TOEFLのような0から1000点満点といったスコアが計算できるのです。

In [3]:
using Interact, Plots, StatsFuns, CSSUtil, ORCA

gr() 
@manipulate for α = 0:0.01:3, β = -4:0.01:4, γ = 0:.01:1
        θ = [-4:0.001:4;]
        prob = γ .+ (1-γ).*logistic.((θ.-β)*α)
        plot(θ, prob, 
        label = "ICC", 
        xlabel = "theta", ylabel = "Correct probability", 
        xlims = (-4, 4), ylims = (0, 1))
    end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope(Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["α"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")))], Dict{Symbol,Any}(:className=>"interact-flex-row-left")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>301,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()))], Dict{Symbol,Any}(:className=>"interact-flex-row-center")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")))], Dict{Symbol,Any}(:className=>"interact-flex-row-right"))], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
151, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/takuizum/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/takuizum/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/takuizum/.julia/packages/InteractBase/l8cEC/src/../assets/all.js"), Asset("css", nothing, "/Users/takuizum/.julia/packages/InteractBase/l8cEC/src/../assets/style.css"), Asset("css", nothing, "/Users/takuizum/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"0.01\",\"0.02\",\"0.03\",\"0.04\",\"0.05\",\"0.06\",\"0.07\",\"0.08\",\"0.09\",\"0.1\",\"0.11\",\"0.12\",\"0.13\",\"0.14\",\"0.15\",\"0.16\",\"0.17\",\"0.18\",\"0.19\",\"0.2\",\"0.21\",\"0.22\",\"0.23\",\"0.24\",\"0.25\",\"0.26\",\"0.27\",\"0.28\",\"0.29\",\"0.3\",\"0.31\",\"0.32\",\"0.33\",\"0.34\",\"0.35\",\"0.36\",\"0.37\",\"0.38\",\"0.39\",\"0.4\",\"0.41\",\"0.42\",\"0.43\",\"0.44\",\"0.45\",\"0.46\",\"0.47\",\"0.48\",\"0.49\",\"0.5\",\"0.51\",\"0.52\",\"0.53\",\"0.54\",\"0.55\",\"0.5

## 等化 (equating) とはなにか

TOEFLのガイドに戻ってみると，統計的な処理＝等化(equating)を行っていると書かれています。この等化とは「異なるテスト得点や問題の難易度を比較可能な形に変換すること」です。等化には大きく二種類あり，素点に基づく等化とIRTに基づく等化があります。

そして結論から言うと，等化の精度(precison)は，後者のほうが高い場合がほとんどなのです。なぜならIRTは素点よりもより多くの統計的な仮定をしいており，それが成り立つという条件下で非常に柔軟な等化が実現できるからです。この等化の原理についてかんたんに説明します。

等化の特徴は「線形変換」です。つまり，あるテストのパラメタ$\beta$を$A\beta+K$という風に変換することです。あるいは$\alpha$を$\frac{1}{A}\alpha$と変換する事に当たります。等化の作業ではこの$A$と$K$をなんらかの方法で推定します（たとえばテスト間で共通する$\beta$の平均と標準偏差から推定する）。実はIRTのパラメタは，様々な要因により，完全に同じ項目を異なる集団に実施した場合，異なる数値のパラメタが推定されてしまいます。これでは一見するとパラメタが比較できないようなのですが，このあとで説明する線形変換を利用して，比較可能になるのです。推定方法を説明すると，やや冗長になりますので，ここでは「なぜ線形変換ができるのか」について詳しく説明します。

IRTの$\theta$と$\beta$に関しては以下の性質が成り立ちます。

$$
\theta = A\theta^\ast+K \\
\beta = A\beta^\ast+K
$$

また，$\alpha$に関しては，
$$
\alpha = \frac{\alpha^\ast}{A}
$$

が成り立ちます。したがって，この3つの変換式を同時に施せば，IRTの反応確率は不変です。

In [4]:
gr() 
@manipulate for A = -10:0.1:10, K = -10:0.1:10
        θ = [-4:0.001:4;]
        α  = 1
        β = 1
        γ = 0
        prob = γ .+ (1-γ).*logistic.(( (θ*A.+K) .- (β*A.+K)).*α./A)
        plot(θ, prob,
        label = "ICC",
        xlabel = "theta", ylabel = "Correct probability",
        xlims = (-4, 4), ylims = (0, 1))
    end
# 最初にAのスライダーを少し動かさないと関数が表示されません。

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope(Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["A"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")))], Dict{Symbol,Any}(:className=>"interact-flex-row-left")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>201,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()))], Dict{Symbol,Any}(:className=>"interact-flex-row-center")), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")))], Dict{Symbol,Any}(:className=>"interact-flex-row-right"))], Dict{Symbol,Any}(:className=>"interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"index"=>(Observable{Any} with 2 listeners. Value:
101, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/takuizum/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/takuizum/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/takuizum/.julia/packages/InteractBase/l8cEC/src/../assets/all.js"), Asset("css", nothing, "/Users/takuizum/.julia/packages/InteractBase/l8cEC/src/../assets/style.css"), Asset("css", nothing, "/Users/takuizum/.julia/packages/Interact/0klKX/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes"=>Any[JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index"=>Any[JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:0), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)), WebIO.JSString[JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-10.0\",\"-9.9\",\"-9.8\",\"-9.7\",\"-9.6\",\"-9.5\",\"-9.4\",\"-9.3\",\"-9.2\",\"-9.1\",\"-9.0\",\"-8.9\",\"-8.8\",\"-8.7\",\"-8.6\",\"-8.5\",\"-8.4\",\"-8.3\",\"-8.2\",\"-8.1\",\"-8.0\",\"-7.9\",\"-7.8\",\"-7.7\",\"-7.6\",\"-7.5\",\"-7.4\",\"-7.3\",\"-7.2\",\"-7.1\",\"-7.0\",\"-6.9\",\"-6.8\",\"-6.7\",\"-6.6\",\"-6.5\",\"-6.4\",\"-6.3\",\"-6.2\",\"-6.1\",\"-6.0\",\"-5.9\",\"-5.8\",\"-5.7\",\"-5.6\",\"-5.5\",\"-5.4\",\"-5.3\",\"-5.2\",\"-5.1\",\"-5.0\",\"-4.9\",\"-4.8\",\"-4.7\",\"-4.6\",\"-4.5\

上のプロットでは，先ほどの式変形をいろんなAとKで行って見た場合を実演できます。どんな組み合わせであってもICCは不変な様子がわかると思います。

したがって，テストAとテストBの一部に同じテスト項目が少数（たとえば10項目）含まれていれば，それらのパラメタをもとにAとKを計算し，残りの共通しない項目を線形変換することで，**理論上は**2つのテストのパラメタを完全に同じパラメタに変換できるのです。

## ヒトはなぜIRTを使うのか 〜それって素点じゃだめなんですか？〜

なぜややこしい計算をして，$\theta$なんてスコアに換算する必要があるのでしょうか。それは素点が抱える項目依存性に大きく関係しています。